In [0]:
import requests
import json
from datetime import datetime,date,timedelta
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType,IntegerType,DateType, NullType
from pyspark.sql.functions import from_json, col, schema_of_json, lit,substring,to_date, split,col,udf,transform,date_format,current_timestamp, to_timestamp, concat

In [0]:
dbutils.widgets.text("volume_path", "")

In [0]:
volume_path=dbutils.widgets.get("volume_path")

In [0]:
# Define the API endpoint and parameters

START_DATE = (date.today() - timedelta(90)).strftime("%Y-%m-%d")
END_DATE = datetime.now().strftime("%Y-%m-%d")

# Configuration - USA only with geographic bounds
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={START_DATE}&endtime={END_DATE}&minlatitude=24.6&maxlatitude=49.0&minlongitude=-125.0&maxlongitude=-65.0"

# -
# SoQL query for filtering crash_date between two dates (inclusive of start, exclusive of end)



try :
    # Make the request
    response = requests.get(url)
    
    # Check and parse the result
    if response.status_code == 200:
        res = response.json().get("features",[])
        print(f"Retrieved {len(res)} records.")
    else:
        print(f"Request failed: {response.status_code}, {response.text}") 

except Exception as e:
    print(f"Error fetching data from API: {e}")
    raise

In [0]:

output_path ="/Volumes/earth_data/bronze/operationaldata/earthquake_data.json"
json_data = json.dumps(res, indent=4)
dbutils.fs.put(output_path, json_data, overwrite=True)

In [0]:
# COMMAND ----------

# df = spark.read.option("multiline", "true").json("/Volumes/earth_data/bronze/operationaldata/earthquake_data.json")